In [6]:
import sys
sys.path.append("../")

import core.optimizers as O
import torch
import core.losses as l
import core.metrics as M
from core.model_structure import Deep_learning_Model
from transformer_operations import *
from transformer import *
class multilayer_transfomer(Deep_learning_Model):
     def __init__(self, optimizer, loss,dmodel,vocab_size):
          super().__init__(optimizer, loss)
          self.decoders=[]
          self.encoders=[]
          self.weights=[]
          self.vocab_size=vocab_size
          self.model_dimensions=dmodel
          self.linear=Linear(dmodel,vocab_size)
          self.weights.extend(self.linear.weights)
          self.embedings_layer=Embdeing_layer(self.model_dimensions)
     def add_encoder(self,num_heads):
          e=transformer_encoder(num_heads,self.model_dimensions)
          self.encoders.append(e)
          self.weights.extend(e.encoder_weights)
     def add_decoder(self,num_heads):
          d=transformer_decoder(num_heads,self.model_dimensions,self.vocab_size)
          self.decoders.append(d)
          self.weights.extend(d.decoder_weights)
     def forward_propagation(self,source,target_embedigns):
          x=source
          for encoder in self.encoders:
               x=encoder.encode(x)
          encoder_res=x
          for decoder in self.decoders:
               encoder_res=decoder.multi_layer_decode(encoder_res,target_embedigns)
          decoder_res=encoder_res
          logits=self.linear.forward(decoder_res)
          return logits
     def  seq_emebidngs(self,src,tgt):
              src_emb = self.embedings_layer.get_embedings(src)
              tgt_emb = self.embedings_layer.get_embedings(tgt[:, :-1])

              tgt_out = tgt[:, 1:]  
              pos_src_emb=pos_encoding(src_emb,self.model_dimensions)
              pos_tgt_emb=pos_encoding(tgt_emb,self.model_dimensions)
              return pos_src_emb,pos_tgt_emb,tgt_out
     
     def train(self, epochs, x_train, y_train, targets, batch_size, learning_rate):
          num_batches = len(x_train) // batch_size
          for epoch in range(epochs):
               epoch_loss = 0
               indices = torch.randperm(len(x_train))
               x_train, y_train, targets = x_train[indices], y_train[indices], targets[indices]
               for i in range(num_batches):

                    # 2. Forward pass (sans clonage inutile)
                    x_batch = x_train[i*batch_size:(i+1)*batch_size]
                    y_batch = y_train[i*batch_size:(i+1)*batch_size]
                    t_batch = targets[i*batch_size:(i+1)*batch_size]

                    train_pred = self.forward_propagation(x_batch, y_batch)
                    
                    # 3. Compute loss
                    loss = self.loss.compute_loss(train_pred, t_batch)
                    
                    # 4. Backward pass
                    loss.backward(retain_graph=True)
                    
                    # 5. Update weights
                    self.backward_propagation(0.01,epoch+1)
                    
                    epoch_loss += loss.item()
               
               print(f'Epoch {epoch+1}, Loss: {epoch_loss/num_batches:.4f}')

               

    
          
          
          
     

     


In [7]:
def generate_clean_data(batch_size, seq_len, vocab_size):
    src = torch.randint(1, vocab_size - seq_len, (batch_size, seq_len))
    tgt = torch.flip(src, dims=[1]) + 1
    return src, tgt


x,y=generate_clean_data(50,7,10)


transfomer_model=multilayer_transfomer("adam","Crossentropy",10,10)
x,y_in,y_out=transfomer_model.seq_emebidngs(x,y)

transfomer_model.add_encoder(
    1
),
transfomer_model.add_encoder(
    1
)
transfomer_model.add_decoder(
    1,
)


y_out=torch.eye(10)[y_out]
y_out.shape


transfomer_model.train(10,x,y_in,y_out,1,0.01)

Epoch 1, Loss: 0.1794
Epoch 2, Loss: 0.1792
Epoch 3, Loss: 0.1791
Epoch 4, Loss: 0.1791
Epoch 5, Loss: 0.1790
Epoch 6, Loss: 0.1789
Epoch 7, Loss: 0.1786
Epoch 8, Loss: 0.1782
Epoch 9, Loss: 0.1779
Epoch 10, Loss: 0.1778
